<h1>Movie Description Text Pre-Processing</h1>

In [1]:
import pandas as pd
import requests
import json
import unidecode
# import goslate
# import sys
# import os
# # project_root = "D:\\Leaning_NLP"
# # sys.path.append(os.path.abspath(os.path.join(os.path.dirname(project_root), 'polyglot')))
# import polyglot
# from polyglot.text import Text
# from transliterate import translit, get_available_language_codes

<h2>Collecting Data</h2>

In [2]:
URL = "https://api.themoviedb.org/3/movie/top_rated?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US&page="
movies = []
for i in range(100): 
    print(f"\rCollecting movies from page : {i+1}", end = '\r')
    lst = json.loads(requests.get(URL + f"{i+1}").text)['results']
    for movie in lst: 
        movies.append(movie)
print("Collection Completed!!        ", end = "\n")    

Collection Completed!!        100


In [3]:
# movies[0]
GENRE_URL = "https://api.themoviedb.org/3/genre/movie/list?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US"
lst = json.loads(requests.get(GENRE_URL).text)['genres']
mp = {}
for item in lst:
    mp[item['id']] = item['name']

In [4]:
lst = []
for movie in movies:
    txt = {}
    txt['original_language'] = movie['original_language']
    txt['original_title'] = movie['original_title']
    # if txt['original_language'] != 'en':
    #     txt['original_title'] = unidecode.unidecode(movie['original_title']).lower()
    txt['overview'] = movie['overview']
    txt['genre'] = []
    for id in movie['genre_ids']:
        txt['genre'].append(mp[id])
    lst.append(txt)
    
df = pd.DataFrame(lst)
df.head(10)

,original_language,original_title,overview,genre
0,en,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,"[Drama, Crime]"
1,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[Drama, Crime]"
2,en,The Godfather Part II,In the continuing saga of the Corleone crime f...,"[Drama, Crime]"
3,en,Schindler's List,The true story of how businessman Oskar Schind...,"[Drama, History, War]"
4,en,12 Angry Men,The defense and the prosecution have rested an...,[Drama]
5,hi,दिलवाले दुल्हनिया ले जायेंगे,"Raj is a rich, carefree, happy-go-lucky second...","[Comedy, Drama, Romance]"
6,ja,千と千尋の神隠し,"A young girl, Chihiro, becomes trapped in a st...","[Animation, Family, Fantasy]"
7,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,"[Drama, Action, Crime, Thriller]"
8,ko,기생충,"All unemployed, Ki-taek's family takes peculia...","[Comedy, Thriller, Drama]"
9,en,The Green Mile,A supernatural tale set on death row in a Sout...,"[Fantasy, Drama, Crime]"


In [5]:
# df['original_language'].value_counts()

In [6]:
for r in range(6):
    print(df['original_title'][r])

The Shawshank Redemption
The Godfather
The Godfather Part II
Schindler's List
12 Angry Men
दिलवाले दुल्हनिया ले जायेंगे


In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

chrome_options = Options()
chrome_options.add_experimental_option("detach",True)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=chrome_options)
driver.maximize_window()


from bs4 import BeautifulSoup
import requests

URL = "https://www.themoviedb.org/search?query="

def convert_english(qury):
    try:
        html_text = requests.get(URL + qury).text
        soup = BeautifulSoup(html_text, 'lxml')
        res_box = soup.find('div', class_ = 'results flex')
        res = res_box.find('div', class_ = 'wrapper')
        res = res.find('a')
        # print(res['href'])
        href = res['href']
        forv_url = "https://www.themoviedb.org/"
        href = forv_url + href
        html_text = requests.get(href).text
        soup = BeautifulSoup(html_text, 'lxml')
        res_lang = soup.find('html')['lang']
        if(res_lang != 'en'):
            driver.get(href)
            # button = driver.find_element('xpath', '/html/body/div[1]/header/div[1]/div/div[2]/ul/li[2]')
            # button.click()
            # button = driver.find_element('xpath', "//*[contains(text(), 'English (en-US)')]")
            html_text = driver.page_source
            soup = BeautifulSoup(html_text, 'lxml')
        res = soup.find('section', class_ = 'header poster')
        res = res.find('h2')
        return res.text
    except:
        print("Issue With", qury)
        return qury
        
# print(convert_english('기생충'))

# df['original_title'] = df.apply(lambda x : convert_english(x['original_title'], x['original_language']), axis = 1)

# for i in range(df.shape[0])
for i in range(100):
    if df['original_language'][i] in ['en', 'fr', 'it', 'es', 'de']:
        continue
    df['original_title'][i] = convert_english(df['original_title'][i])

df.head(10)

driver.quit()

C:\Users\HP\AppData\Local\Temp\ipykernel_20584\540118842.py:56: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['original_title'][i] = convert_english(df['original_title'][i])


In [8]:
for i in range(df.shape[0]):
    df.iloc[i, 1] = df.iloc[i, 1].replace('\n', " ")
df.head(10)

,original_language,original_title,overview,genre
0,en,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,"[Drama, Crime]"
1,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[Drama, Crime]"
2,en,The Godfather Part II,In the continuing saga of the Corleone crime f...,"[Drama, Crime]"
3,en,Schindler's List,The true story of how businessman Oskar Schind...,"[Drama, History, War]"
4,en,12 Angry Men,The defense and the prosecution have rested an...,[Drama]
5,hi,Dilwale Dulhania Le Jayenge (1995),"Raj is a rich, carefree, happy-go-lucky second...","[Comedy, Drama, Romance]"
6,ja,Spirited Away (2001),"A young girl, Chihiro, becomes trapped in a st...","[Animation, Family, Fantasy]"
7,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,"[Drama, Action, Crime, Thriller]"
8,ko,Parasite (2019),"All unemployed, Ki-taek's family takes peculia...","[Comedy, Thriller, Drama]"
9,en,The Green Mile,A supernatural tale set on death row in a Sout...,"[Fantasy, Drama, Crime]"


<!-- ![alt text]("Screenshot 2024-06-09 203944.png") -->
<h2>PreProcessing Text</h2>
<h4>We will do preprocessing of data on the overview tag</h4>
<h4>Method Used:</h4>
<img src = "Screenshot 2024-06-09 203944.png">

<h3>Lowercasing</h3>

In [9]:
df['overview'] = df['overview'].apply(lambda x : x.lower())
df.sample(10)

,original_language,original_title,overview,genre
501,en,Emancipation,inspired by the gripping true story of a man w...,"[Drama, War, History]"
942,ja,ゴジラ-1.0,"in postwar japan, godzilla brings new devastat...","[Science Fiction, Horror, Action]"
1197,fr,Seul contre tous,after completing jail time for beating up a ma...,"[Drama, Thriller]"
1635,en,Sicario,an idealistic fbi agent is enlisted by a gover...,"[Action, Crime, Thriller]"
305,cn,重慶森林,two melancholic hong kong policemen fall in lo...,"[Drama, Comedy, Romance]"
1880,en,Dog,an army ranger and his dog embark on a road tr...,"[Drama, Comedy]"
1388,en,Extraction 2,tasked with extracting a family who is at the ...,"[Action, Thriller]"
1155,en,Little Big Man,"jack crabb, looking back from extreme old age,...","[Adventure, Western, Comedy, Drama]"
1044,en,Mary Poppins,mr banks is looking for a nanny for his two mi...,"[Comedy, Family, Fantasy]"
1296,en,In Bruges,"ray and ken, two hit men, are in bruges, belgi...","[Comedy, Drama, Crime]"


<h3>Remove HTML tags</h3>

In [10]:
import re
def tags_remover(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)

df['overview'] = df['overview'].apply(lambda x : tags_remover(x))
df.sample(10)

,original_language,original_title,overview,genre
437,en,Finch,"on a post-apocalyptic earth, a robot, built to...","[Science Fiction, Drama, Adventure]"
546,ja,千年女優,documentary filmmaker genya tachibana has trac...,"[Animation, Drama, Romance, Fantasy]"
688,en,Shrek,it ain't easy bein' green -- especially if you...,"[Animation, Comedy, Fantasy, Adventure, Family]"
542,en,How to Train Your Dragon,as the son of a viking leader on the cusp of m...,"[Fantasy, Adventure, Animation, Family]"
1577,fr,Sauver ou périr,franck is a firefighter in paris. he saves peo...,[Drama]
728,fr,Divines,in a ghetto where religion and drug traffickin...,[Drama]
204,en,Eternal Sunshine of the Spotless Mind,"joel barish, heartbroken that his girlfriend u...","[Science Fiction, Drama, Romance]"
1246,fr,Edmond,"paris, france, december 1897. the young playwr...","[Comedy, History]"
1919,en,RoboCop,"in a violent, near-apocalyptic detroit, evil c...","[Action, Thriller, Science Fiction]"
1487,en,Wind River,an fbi agent teams with the town's veteran gam...,"[Crime, Drama, Mystery, Thriller]"


<h3>Remove URLs </h3>

In [11]:
def remove_urls(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

df['overview'] = df['overview'].apply(remove_urls)
df.sample(10)

,original_language,original_title,overview,genre
1000,en,I Origins,a molecular biologist's study of the human eye...,"[Science Fiction, Drama]"
1656,en,The Peanut Butter Falcon,a down-on-his-luck crab fisherman embarks on a...,"[Adventure, Comedy, Drama]"
1170,en,The Revenant,"in the 1820s, a frontiersman, hugh glass, sets...","[Western, Drama, Adventure]"
359,en,CODA,"as a coda (child of deaf adults), ruby is the ...","[Drama, Music, Romance]"
739,en,Holding the Man,tim and john fell in love while teenagers at t...,[Drama]
1714,en,Pelé: Birth of a Legend,"the life story of brazilian football legend, p...",[Drama]
1314,en,The Royal Tenenbaums,royal tenenbaum and his wife etheline had thre...,"[Comedy, Drama]"
1290,en,A Quiet Place Part II,"following the events at home, the abbott famil...","[Science Fiction, Thriller, Horror]"
970,en,Tangled,when the kingdom's most wanted-and most charmi...,"[Animation, Family]"
1454,en,The Last Duel,king charles vi declares that knight jean de c...,"[History, Drama, Action]"


<h3>Remove Punctuation</h3>

In [24]:
# punctuation_marks = '"#$%&\'()*+,-./:;<=>@[\\]^_`{|}~'
import string

punctuation_marks = string.punctuation

# We excluded '!' & '@' from punctuations as it could give useful information. 
punctuation_marks = punctuation_marks[1:20] + punctuation_marks[21:]

def remove_punctuations(text):
    return text.translate(str.maketrans(' ', ' ', punctuation_marks))

df['overview'] = df['overview'].apply(remove_punctuations)
df.head(10)

'"#$%&\'()*+,-./:;<=>@[\\]^_`{|}~'

<h3>Chat Word Treatment</h3>

In [34]:
import csv

# Git hub link for the same can be found here : https://github.com/rishabhverma17/sms_slang_translator
def chat_word_converter(user_string):
    user_string = user_string.split(" ")
    j = 0
    for _str in user_string:
        # File path which consists of Abbreviations.
        fileName = "slang.txt"
        # File Access mode [Read Mode]
        accessMode = "r"
        with open(fileName, accessMode) as myCSVfile:
            # Reading file as CSV with delimiter as "=", so that abbreviation are stored in row[0] and phrases in row[1]
            dataFromFile = csv.reader(myCSVfile, delimiter="=")
            # Removing Special Characters.
            _str = re.sub('[^a-zA-Z0-9-_.]', '', _str)
            for row in dataFromFile:
                # Check if selected word matches short forms[LHS] in text file.
                if _str.upper() == row[0]:
                    # If match found replace it with its Abbreviation in text file.
                    user_string[j] = row[1]
            myCSVfile.close()
        j = j + 1
    # Replacing commas with spaces for final output.
    return ' '.join(user_string).lower()

# print(chat_word_converter("Hi bro ltns"))

df['overview'] = df['overview'].apply(chat_word_converter)
df.sample(5)

,original_language,original_title,overview,genre
1099,en,Shang-Chi and the Legend of the Ten Rings,shangchi must confront the past he thought he ...,"[Action, Adventure, Fantasy]"
126,en,Hacksaw Ridge,wwii american army medic desmond t doss who se...,"[Drama, History, War]"
1970,en,Crossroads,a wannabe blues guitar virtuoso seeks a longlo...,"[Drama, Music, Mystery, Romance]"
979,ja,ストレンヂア -無皇刃譚-,pursued by formidable chinese assassins young ...,"[Action, Animation, History]"
666,en,A Woman Under the Influence,mabel longhetti desperate and lonely is marrie...,[Drama]


<h3>Spelling Correction</h3>

In [57]:
from textblob import TextBlob
from spellchecker import SpellChecker
spell = SpellChecker()

def spelling_correction(text):
    new_text = TextBlob(text)
    return new_text.correct()

print(spelling_correction("gunda"))

guns
